Compact pathway creator function

Defines all pathways and returns a model with the selected pathway added.
The function can be called alone or get assigned to a new variable, but it always alters the handed model. In order to not touch manipulate the existing model work within with model: or create a copy of your model with model.copy() before handing that copy to this function.

Function

create_pathway(model,path_selector)

model ... needs to be handed a model, since it doesnt import it itself to save time when function if called multiple times
path_selector ... determines which path is added
Import the function into your script with the following code, thic function script has to be in the same folder.

%run -i create_pathway.ipynb

Pathway legend

0 -> no additional reactions
1 -> NAM pathway
2 -> NA pathway
3 -> NR pathway

path_name = ['no additional reactions','NAM Pathway','NA Pathway','NR Pathway']

Huang, Zhongshi, et al. describes 3 different pathways to produce NMN in *E.coli*, which can be seen in figure 2. The 3 pathways generate MNM accumulatively mainly through the two metabolites NaMN (nicotinic acid mononucleotide) and (NAM: nicotinamide), which are direct precursors for NMN. The reactions, enzymes as well as corresponding metabolites or cofactors were investigated using the Kyoto Encyclopedia of Genes and Genomes KEGG database. To receive a similar strain like Huang, Zhongshi, et al. the regulatory genes nadR, pncC, ushA, and purR must be knocked out. This is especially important for route 3.

NAM Pathway

This pathway contains out of 3 steps: one transportation and two reactions. First, the precursor NAM will be transported into the cell through the NiaP transporter. Secondly Ribose-5-phosphate is converted through the PRPP synthase to PRPP. Thirdly the heterologous nicotinamide phosphoribosyl transferase will use the precursor PRPP and NAM and produce the compound NMN.


In [2]:
# Importing used packages and model
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
model = read_sbml_model('iML1515.xml')

# Add metabolite
model.add_metabolites([
    Metabolite('ncam_e', name='nicotineamide', compartment='e', formula = 'C6H6N2O'  ),
    Metabolite('na_c', name='Nicotinic acid',  compartment='c', formula = 'C6H5NO2' ),
    Metabolite('namn_c', name='nicotinic acid mononucleotide',  compartment='c', formula = 'C11H14NO9P' ),
    Metabolite('nr_c', name='nicotineamide riboside',  compartment='c', formula = 'C11H15N2O5' ),
    Metabolite('nr_e', name='nicotineamide riboside',  compartment='e',formula = 'C11H15N2O5' ),
    Metabolite('na_e', name='Nicotinic acid',  compartment='e', formula = 'C6H5NO2' ),
])

In [3]:
# NAM Pathway
nico_phos_trans = Reaction('NAMPT')
nico_phos_trans.name = 'nicotinamide-phosphoribosyl transferase'
# NAM + PRPP <=> NMN + PPi
# NMN = Metabolite (id='nmn_c', compartment = 'c')
# NMN.name = 'Nicotinamide mononucleotide'
# NMN.formula = 'C11H15N2O8P'

nico_phos_trans.add_metabolites (({model.metabolites.prpp_c: -1, 
                                 model.metabolites.ncam_c: -1,
                                 model.metabolites.nmn_c: 1,
                                 model.metabolites.ppi_c: 1}))

R5P_prpp=Reaction('BaPRS')
R5P_prpp.name= ('Ribose 5 phosphate to PRPP')
R5P_prpp.add_metabolites (({model.metabolites.prpp_c: 1, 
                                 model.metabolites.r5p_c: -1}))

# Ex_NAM -> NAM
nicotine_trans = Reaction('NiaP')
nicotine_trans.name = 'nicotine transporter'

nicotine_trans.add_metabolites (({model.metabolites.ncam_c: 1, 
                                  model.metabolites.ncam_e: -1}))

In [4]:
# Set up pathway
nam_path = [nico_phos_trans,nicotine_trans,R5P_prpp]

The second pathway simply starts with nicotinic acid as a substrate, which is then converted by the homogenous nicotinic acid phosphoribosyl transferase (EcpncB) to NaMN. The heterologous NMN synthase (FtNadE) translates NAMN afterwards to NMN. 

In [5]:
# NA Pathway

# NA <=> NAMN 
nico_acid_phosp_trans = Reaction ('PNCB')
nico_acid_phosp_trans.name = 'nicotinicacid-phosphoribosyl transferase'

nico_acid_phosp_trans.add_metabolites  (({model.metabolites.na_c: -1,
                                          model.metabolites.namn_c: 1})) 
#NAMN <=> NMN 
nico_syntase= Reaction ('FtNadE')
nico_syntase.name = 'nicotineamide mononucleotide synthetase'
                                        
nico_syntase.add_metabolites (({model.metabolites.namn_c:-1,
                                model.metabolites.nmn_c: 1})) 
#EX_NA -> NA
nicotinic_acid_trans = Reaction('NiaP2')
nicotinic_acid_trans.name = 'nicotinic acid transporter'

nicotinic_acid_trans.add_metabolites (({model.metabolites.na_c: 1, 
                                  model.metabolites.na_e: -1}))

In [6]:
# set up pathway 
na_path = [nico_acid_phosp_trans,nico_syntase,nicotinic_acid_trans ]


NR Pathway 
The third pathway requires the most steps and is considered as the most expensive one, due to the cost of the substrate nicotinamide riboside (NR). As soon NR enters the cell it is converted by 2 enzymes named purine-nucleoside phosphorylase and pncA to NAM. From there it follows the same path as described in route 2.


In [7]:
# NR Pathway
#NR to NAM 
Pur_nuc_phosph = Reaction ('DeoD')
Pur_nuc_phosph.name = 'purine-nucleoside phosphorylase'

Pur_nuc_phosph.add_metabolites  (({model.metabolites.nr_c: -1,
                                 model.metabolites.ncam_c: 1})) 

# NAM to NA 
Pyr_dase = Reaction ('PncA')
Pyr_dase.name = 'Pyrazinamidease' 

Pyr_dase.add_metabolites  (({model.metabolites.ncam_c: -1,
                                model.metabolites.na_c: 1})) 

#EX_NR -> NR
nicotineamide_riboside_trans = Reaction('PnuC')
nicotineamide_riboside_trans.name = 'nicotineamide riboside transporter'

nicotineamide_riboside_trans.add_metabolites (({model.metabolites.nr_c: 1, 
                                  model.metabolites.nr_e: -1}))

In [8]:
nr_path = [Pur_nuc_phosph,Pyr_dase,nicotineamide_riboside_trans]

Transport for NMN 

bla bla bla 

In [9]:
# Transport reaction cytosol -> extracellular 

nmn_ctoe = Reaction('BMpnuC')
nmn_ctoe.name = 'NMN transporter'
nmn_ctoe.add_metabolites({ model.metabolites.nmn_c: -1,
                          model.metabolites.nmn_e: 1})
# Removal reaction -> alernativ implementation as boundary within the create_pathway function

# set up transport path
transport = [nmn_ctoe]

#model.add_boundary(model.metabolites.nmn_c, type='demand')

### Adding selected pathway to model

In [10]:
pathways = [[], nam_path, na_path, nr_path]

In [11]:
def create_pathway(model, path_selector=0):
    model.add_reactions(pathways[path_selector])
    model.add_reactions(transport)
    return model